In [5]:
import pandas as pd
import os

In [9]:
if not os.path.exists('danone.pickle'):
    url = 'https://challenges-asset-files.s3.us-east-2.amazonaws.com/Events/Danone+Hackathon+Mexico/data/train_products.json'
    df = pd.read_json(url, orient='index')
    df.to_pickle('danone.pickle')
    # s = df.size
    # m = df.memory_usage(deep=True)
    # print(s, m.sum()/(1024*1024))
else:
    df=pd.read_pickle('danone.pickle')

In [19]:
df.columns

Index(['name', 'brand', 'generic_name', 'categories_hierarchy', 'is_beverage',
       'selling_countries', 'ingredient_origins', 'ingredients',
       'additives_count', 'calcium_100g', 'carbohydrates_100g',
       'energy_kcal_100g', 'fat_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'sodium_100g', 'sugars_100g', 'nutrition_grade',
       'packaging_materials',
       'non_recyclable_and_non_biodegradable_materials_count',
       'est_co2_agriculture', 'est_co2_consumption', 'est_co2_distribution',
       'est_co2_packaging', 'est_co2_processing', 'est_co2_transportation',
       'ecoscore_grade'],
      dtype='object')

In [24]:
df.plot(
    x='est_co2_agriculture',
    y='ecoscore_grade',
    kind='kde'
    )